In [2]:
#пробегаю по всем кейсам проекта петрель, чтобы найти нужные гриды

import os
import shutil


#path = "C:\Users\ShishaevGY\Documents\Python Scripts\Anomaly_detection_STC\Clean_with_window_final\DATA\plot_all_well_evol" #error in path
dst = "../DATA/INIT_DATA/" #куда копировать нужные гриды
word = '_PERMX' #какой грид искать
for root, dirs, files in os.walk(path):
    for _file in files:
        if word in _file:
            #print (root + '\\' + _file)
            shutil.copy2(root + '\\' + _file, dst)

In [5]:
#убираем шапку из всех гридов, оставляя только сами гриды, без информационной шапки
import os
import shutil

main_path_to_data = "../DATA/INIT_DATA/"

word = '_PERMX' #какой грид искать

clean_data_path = "../DATA/" + word
#удаляем шапку из файла с гридами
i = 0


os.makedirs(clean_data_path)

for root, dirs, files in os.walk(main_path_to_data):
    for _file in files:
        if word in _file:
            with open(root + '\\' + _file, "r") as f:
                lines = f.readlines()
            with open(clean_data_path + '\\' + _file, "w") as f:
                for line in lines:
                    if i > 11:
                        f.write(line)
                    i += 1
        i = 0

In [6]:
import os
import shutil

def Grid_Reader(clean_data_path, word, _file, i_max, j_max, k_max):
#читаем файл с гридами и записываем в массив

    import pandas as pd
    import numpy as np
    
    base = open(clean_data_path + word + '\\' + _file, "r") #считываю инф с файла
    print (base)
    digit = []

    #читаем файл и разбиваем его на отдельные "слова"
    for line in base:
        words = line.split()
        for word in words:
            if '*' in word:
                x,n = list(map(float, word.split('*')))
                for i in range(int(x)):
                    digit.append(n)
            else:
                word = word.split()
                if '/' in word:
                    continue
                int_lst = [float(x) for x in word]
                digit.append(int_lst)
            
    #переводим лист "digit" в формат float. пока одномерный
    One_Dim = np.empty(len(digit))

    j=0
    for i in digit:
        if type(i) == list:
            for x in i:
                One_Dim[j] = float(x)
        else:
            One_Dim[j] = i
        j +=1
    print('One_Dim', One_Dim)

    
    #проверка размера массива и размерности грида        
    if i_max*j_max*k_max != len(One_Dim):
        print('wrong array size!!!')

    #One_Dim = One_Dim.reshape((-1,1))
    return One_Dim

In [7]:
#забиваем в csv файл все гриды
import pandas as pd
import numpy as np
import os
import shutil


word = '_PERMX' #какой грид искать
clean_data_path = "../DATA/"
i_max = 24 # X grid size
j_max = 18 # Y grid size
k_max = 15 # Z grid size

col_names = []

dataset = pd.DataFrame(columns=col_names)

for root, dirs, files in os.walk(clean_data_path + word):
    for _file in files:
        if word in _file:
            dataset[_file] = Grid_Reader(clean_data_path, word, _file, i_max, j_max, k_max)

file_name = word+'.csv'    
pd.DataFrame(dataset).to_csv(clean_data_path+file_name, encoding='utf-8', index=False)


<_io.TextIOWrapper name='../DATA/_PERMX\\BASE_CASE_1000_PROP_PERMX.GRDECL' mode='r' encoding='cp1251'>
One_Dim [159.255  170.404   89.3909 ...  65.2549  62.7315  68.8542]


ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [3]:
#читаем одномерный массив и переводим его в 3Д размерность, как в гриде.

main_path_to_data = "../DATA/"

# задаем размерность грида
i_max = 24
j_max = 18
k_max = 15

#приводим массив к 3D размерности грида.    
Three_Dim = np.array(np.zeros((i_max, j_max, k_max)))
i = 0
j = 0
k = 0
z = 0

df = pd.read_csv(main_path_to_data+'xxx.csv')
One_Dim = df.values

while k <= k_max-1:
    while j <= j_max-1:
        while i <= i_max-1:
            Three_Dim[i, j ,k] = One_Dim[z]
            i += 1
            z += 1
        i = 0
        j += 1
    j = 0
    k += 1

print (Three_Dim)

[[[ 87.386    36.7012   96.704   ...  15.2026   20.1017   16.04   ]
  [ 36.764    25.307    96.8363  ...  12.4688   28.8132   14.9288 ]
  [ 36.6562   32.9808   69.8601  ...   8.13058  37.9288   30.9566 ]
  ...
  [ 56.426   227.71     74.4164  ...  38.7076   94.0196  155.187  ]
  [ 35.7288  274.399   111.237   ...  28.1612   82.0534  196.254  ]
  [ 35.5472  137.9      82.88    ...  33.5882  112.127   123.404  ]]

 [[ 93.5038   30.3057   94.5656  ...  27.7153   26.3065    8.8938 ]
  [ 36.6532   34.7464  102.371   ...  14.0153   33.8801   16.6633 ]
  [ 19.207    36.5835   85.6964  ...   9.49557  69.2748   38.0307 ]
  ...
  [ 37.4682  228.194    68.8787  ...  36.0927   36.0111  198.368  ]
  [ 21.9926  184.151    60.2622  ...  66.2955   51.93    194.44   ]
  [ 25.5052  108.286    62.7198  ...  40.0947   62.1425  167.885  ]]

 [[ 49.0505   14.658   121.565   ...  15.7418    0.       11.5117 ]
  [ 36.6752   26.931   115.326   ...  14.205    37.8498   12.7559 ]
  [ 33.0737   35.2964  156.953  